You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [246]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
#import time
#start = time.time()
import pandas as pd
import numpy as np
import re

In [257]:
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression = 'gzip', index_col = 'User-ID', encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'int',
                                                                                                                                                                                         'Location': 'string',
                                                                                                                                                                                         'Age': 'Int64'
                                                                                                                                                                                        })
users.shape

(278858, 2)

In [3]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ';', dtype = {'ISBN': 'string',
                                                                                                                                                                                      'Book-title': 'string',
                                                                                                                                                                                      'Book-Author': 'string',
                                                                                                                                                                                      'Year-Of-Publication': 'string',
                                                                                                                                                                                      'Publisher': 'string',
                                                                                                                                                                                      'Image-URL-S': 'string',
                                                                                                                                                                                      'Image-URL-M': 'string',
                                                                                                                                                                                      'Image-URL-L': 'string',
                                                                                                                                                                                     })
books.shape

# modificare year in integer (ci sono 3 righe problematiche)
# books[books['Year-Of-Publication'].str.contains('\D+')]
# index_col = 'ISBN'

(271359, 8)

In [251]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression = 'gzip', index_col = ['User-ID', 'ISBN'], encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'int',
                                                                                                                                                                                                                 'ISBN': 'string',
                                                                                                                                                                                                                 'Book-Rating': 'int'
                                                                                                                                                                                                                })
book_ratings.shape

# 0 significa che non hanno votato?

(1149780, 1)

In [260]:
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression = 'gzip', index_col = ['user_id', 'book_id'], encoding = 'latin-1', sep = ',', dtype = {'user_id': 'int',
                                                                                                                                                                                                                                'book_id': 'int',
                                                                                                                                                                                                                                'rating': 'int'
                                                                                                                                                                                                                            })
goodbooks_ratings.shape

(99, 1)

In [237]:
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression = 'gzip', index_col = 'book_id', encoding = 'latin-1', sep = ',', dtype = {'book_id': 'int',
                                                                                                                                                                                                 'goodreads_book_id': 'int',
                                                                                                                                                                                                 'best_book_id': 'int',
                                                                                                                                                                                                 'work_id': 'int',
                                                                                                                                                                                                 'books_count': 'int',
                                                                                                                                                                                                 'isbn': 'string',
                                                                                                                                                                                                 'isbn13': 'string',
                                                                                                                                                                                                 'authors': 'string',
                                                                                                                                                                                                 'original_publication_year': 'int',
                                                                                                                                                                                                 'original_title': 'string',
                                                                                                                                                                                                 'title': 'string',
                                                                                                                                                                                                 'language_code': 'string',
                                                                                                                                                                                                 'average_rating': 'double',
                                                                                                                                                                                                 'ratings_count': 'int',
                                                                                                                                                                                                 'work_ratings_count': 'int',
                                                                                                                                                                                                 'work_text_reviews_count': 'int',
                                                                                                                                                                                                 'ratings_1': 'int',
                                                                                                                                                                                                 'ratings_2': 'int',
                                                                                                                                                                                                 'ratings_3': 'int',
                                                                                                                                                                                                 'ratings_4': 'int',
                                                                                                                                                                                                 'ratings_5': 'int',
                                                                                                                                                                                                 'image_url': 'string',
                                                                                                                                                                                                 'small_image_url': 'string'
                                                                                                                                                                                                 })
goodbooks.shape

(99, 22)

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [262]:
users[['City', 'Region', 'Country']] = users['Location'].str.rsplit(',', 2, expand=True)
# Da sistemare country usando le probabilitá
# creare lista 
# sistemare encoding (riga 1318)
users.head()

,Location,Age,City,Region,Country
User-ID,,,,,
1,"nyc, new york, usa",<NA>,nyc,new york,usa
2,"stockton, california, usa",18,stockton,california,usa
3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia
4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal
5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom


### 2. For each book in the *Books* dataset, compute its average rating.

In [263]:
books_book_ratings = pd.merge(books, book_ratings, on = 'ISBN', how = 'left')
books_book_ratings.groupby('ISBN').mean('Book-Rating')

# ci sono valori NaN?
# gli 0 vanno mediati?

,Book-Rating
ISBN,
0000913154,8.0
0001010565,0.0
0001046438,9.0
0001046713,0.0
000104687X,6.0
...,...
B000234N76,0.0
B000234NC6,0.0
B00029DGGO,0.0


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [269]:
goodbooks[['isbn', 'average_rating']].head()

# é giusto il valore avg rating o va ricalcolato dalle stelle?
# bisogna fare una media pesata con altri voti?

,isbn,average_rating
book_id,,
1,439023483,4.34
2,439554934,4.44
3,316015849,3.57
4,61120081,4.25
5,743273567,3.89


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [5]:
merged_books = books.groupby(['Book-Title', 'Book-Author', 'Publisher']).count()
merged_books = merged_books[merged_books['ISBN']>1]
merged_books.head()

# .loc o fancy indexing?

,,,ISBN,Year-Of-Publication,Image-URL-S,Image-URL-M,Image-URL-L
Book-Title,Book-Author,Publisher,,,,,
!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,2,2,2,2
'A Hell of a Place to Lose a Cow': An American Hitchhiking Odyssey,Tim Brookes,National Geographic,2,2,2,2,2
"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,2,2,2,2
101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3,3,3,3,3
15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2,2,2,2,2


### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

### 11. Find the books that appear only in the goodbooks datasets.

### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

### 13. Find the author with the highest average rating according to the goodbooks datasets.